In [ ]:
# %rm -rf phase2_module
# !git clone https://github.com/LgDNet/phase2_module.git
# !cd phase2_module && make copy

In [1]:
cd ..

/home/elicer


/home/elicer/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from phase2_module.src.utils.dataset import Data
from phase2_module.src.models.classification.models import Model
from phase2_module.src.base import BasePiepline
# from phase2_module.src.hyper_parameters.params_optimization import xgboost_opt, set_params_optimization_data
from phase2_module.src.utils.set_seed import seed_everything
from phase2_module.src.utils.top_score_instance import check_the_score
from phase2_module.src.encoder.encoders import Encode

from phase2_module.src.utils.manage_pkl_files import PickleManager as pkl_manager
from phase2_module.src import preprocess
seed_everything()  # NOTE: default 42
_model = Model()

In [3]:
import pandas as pd
import numpy as np
import re, pickle
import warnings
from tqdm import tqdm
warnings.filterwarnings(action='ignore')
def set_up(mode= None):
    pd.set_option('display.max_columns', mode)
set_up()

# Preprocessing

In [4]:
basic = preprocess.Basic
country = preprocess.Country
customer = preprocess.Customer
inquiry = preprocess.Inquiry
product_category = preprocess.ProductCategory
convert_ratio = preprocess.Convert_ratio
customer2 = preprocess.Customer2
category = preprocess.Categorization

# Main

In [5]:
pkls = pkl_manager.map(pkl_manager.loads, pkl_manager.metadata_directory)

runtime_instance = {
    "Train": Data.train,
    "Test": Data.test,
    "Instances":[
        {"instance": basic, "method": ['drop_duplicated'], "params": False},
        {"instance": country, "method": ['apply'], "params": pkls.get("country")},
        {"instance": inquiry, "method": ["apply"], "params": False},
        {"instance": product_category, "method": ["product_categories", 'fill_odds'], "params": {**pkls.get("product_category"), **pkls.get("product_subcategory")}},
        {"instance": customer, "method": ["apply"], "params": pkls.get("customer")},
        {"instance": convert_ratio, "method": ["lead_owner"], "params": {**pkls.get("lead_owner"), **pkls.get("customer_idx")}},
#         {"instance": basic, "method": ['customer_idx_merge_enterprise'], "params": False},
        {"instance": customer2, "method": ['seniority_level', 'job_function', 'customer_type'], "params": pkls.get("customer2")},
        {"instance": category, "method": ['cat_customer_idx','customer_idx_merge_enterprise','historical_cnt_mean'], "params": False},
    ]
}

## preprocessing

In [19]:
df_train = preprocess.runner('Train',**runtime_instance)
df_test = preprocess.runner('Test',**runtime_instance)

In [21]:
cond = df_train['is_converted'] == True
true_num = df_train[cond]['lead_desc_length'].mean()
cond1 = df_train['lead_desc_length'] > true_num
df_train.loc[cond1,'less_timeline'] += 1

## encoding

In [24]:
encode = Encode()

In [25]:
exisiting_columns = [
                     'business_area','business_subarea','business_unit',
                        'enterprise',
#                         'response_corporate',#"customer_country", # country
#                         'customer_type','customer_job','customer_position', #customer
#                         'inquiry_type','expected_timeline', # inquiry
#                         'product_subcategory','product_category','product_modelname' # product_category
                       ] 
preprocess_columns = [
                     'country','continent', # country
                     'customer_type2','job_function','seniority_level', # customer
                     'new_expected_timeline', 'new_inquiry_type',# inquiry
                     'category_2','category_3','category_1',
                     'new_customer_type1','new_customer_type2' # product_category
#                      'test2'
                     ]

label_encode_columns = exisiting_columns + preprocess_columns

In [26]:
# 결측치 채우기
df_train = preprocess.fillna(df_train)
df_test = preprocess.fillna(df_test)

In [27]:
# # one-hot encoding
# onehot_encode_columns = ['category_1']
# df_train = encode.onehot_encoder(df_train,onehot_encode_columns)
# df_test = encode.onehot_encoder(df_test,onehot_encode_columns)

In [28]:
# label encoding
encode.set_up() # 전역 변수 초기화
df_train = encode.label_encoder(df_train,label_encode_columns)
df_test = encode.label_encoder(df_test,label_encode_columns)

# Modeling

## drop column
모델링 전이 column을 drop하면서 다루기가 쉽다고 판단하였음

In [29]:
drop_col = [
            'id_strategic_ver','it_strategic_ver','idit_strategic_ver','ver_pro','ver_cus',
            'customer_country.1',
            "customer_country",#'response_corporate' # country
            'customer_type','customer_job','customer_position', # customer
            'inquiry_type', 'expected_timeline', #inquiry
            'product_subcategory','product_category','product_modelname', # product_category
#             'cate_is_nan', # product_category
    
#             'com_reg_ver_win_rate','idit_strategic_ver','ver_cus',
#             'ver_pro','ver_win_rate_x','ver_win_ratio_per_bu',
            'customer_idx',
            'lead_owner','historial_existing_mean','response_corporate',#'new_customer_idx'
            'customer_type2',
#             'customer_idx_converted_ratio', # ratio
            #'lead_owner', 'lead_owner_converted_ratio', 'lead_owner_customer_idx_mean', # lead_owner
            ]
df_train.drop(drop_col,axis = 1, inplace = True)
df_test.drop(drop_col,axis = 1, inplace = True)

## Scaler

In [30]:
import numpy as np
# log 변환
df_train['lead_desc_length'] = np.log1p(df_train['lead_desc_length'])
df_test['lead_desc_length'] = np.log1p(df_test['lead_desc_length'])

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train[['lead_owner_converted_ratio']] = scaler.fit_transform(df_train[['lead_owner_converted_ratio']])
df_test[['lead_owner_converted_ratio']] = scaler.transform(df_test[['lead_owner_converted_ratio']])

## type

In [ ]:
df_train['new_customer_idx'] = df_train['new_customer_idx'].astype(int)
df_test['new_customer_idx'] = df_test['new_customer_idx'].astype(int)

## train

### catboost

In [32]:
import math
from six.moves import xrange

class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)
        
        result = []
        for index in range(len(targets)):
            e = np.exp(approxes[index])
            p = e / (1 + e)
            der1 = targets[index] - p
            der2 = -p * (1 - p)
            

            if (targets[index] == 0.0) & (p<0.5):
                #( True Negative )
                der1 = (p)*(targets[index] - p)
                der2 = (1-2*p)*(-p * (1 - p))
                
            elif (targets[index] == 0.0) & (p>=0.5):
                # 타겟값이 노말 유저인데,fraud 라고 예측한 경우 ( False Positive )
                der1 = 3*(p)*(targets[index] - p)
                der2 = 3*(1-2*p)*(-p * (1 - p))

            elif (targets[index] > 0.0) :
                # 타겟 값이 fraud 이라면 p값이 높을수록 (정답에 가까울수록) 페널티가 적도록 세팅. 추가로 3배 페널티
                der1 = 3*(1-p)*(targets[index] - p)
                der2 = 3*(2*p-1)*(-p * (1 - p))
                
            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result

In [33]:
model = _model.catboost
model.set_params(iterations=5000,
                depth=12,
                thread_count=3,
                learning_rate=1e-3,
                loss_function=LoglossObjective(),
                eval_metric='PRAUC',
                early_stopping_rounds=400,
                verbose=1000)

### decision_tree

In [318]:
dict_ = {'criterion': 'log_loss', 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 2, 'random_state': 42}
model = _model.decision_tree
model.set_params(**dict_)

DecisionTreeClassifier(criterion='log_loss', max_depth=20, min_samples_leaf=2,
                       min_samples_split=7, random_state=42)

### xgboost

In [238]:
model = _model.xgboost
model.set_params(**{'n_estimators': 2393,
 'max_depth': 16,
 'min_child_weight': 2,
 'learning_rate': 0.06783336462585174,
 'subsample': 0.9511145375386896,
 'gamma': 0.16846211770550093,
 'colsample_bytree': 0.6662896237544925,
 'colsample_bylevel': 0.8781042717330898,
 'colsample_bynode': 0.741271661588135,
'random_state':42})
# model.set_params(random_state=42)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.8781042717330898,
              colsample_bynode=0.741271661588135,
              colsample_bytree=0.6662896237544925, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.16846211770550093,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.06783336462585174,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=16, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=2393, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

### light_gbm

In [ ]:
model = _model.lightgbm
model.set_params(
    num_iterations = 5000,
    objective = 'binary',
    is_unbalance=True,
    max_depth= -1,
    early_stopping_rounds = 500,
   # loss_function=LoglossObjective(),
   # neg_bagging_fraction = 0.1,
    #pos_bagging_fraction = 1.0,
    verbose =0,
    seed = 42)
model.fit(x_train, y_train,eval_set=[(x_test, y_test)],eval_metric = LoglossObjective(),)

### random_state

In [ ]:
model = _model.random_forest
model.set_params(**{'max_depth': 19, 'max_leaf_nodes': 749, 'n_estimators': 349,'random_state':42})

### train_

In [584]:
X = df_train.drop(["is_converted"],axis = 1)
Y = df_train['is_converted']
# Y = np.where(Y == False, 0, 1) # catboost

In [374]:
# oversampling
from imblearn.over_sampling import SMOTE, ADASYN
adasyn = SMOTE(sampling_strategy=0.2,random_state=42)
X, Y = adasyn.fit_resample(X, Y)

In [496]:
def score_print(result_dict):
    for name, score_list in result_dict.items():
        print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

In [586]:
from collections import defaultdict
from sklearn.model_selection import StratifiedKFold
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_result, valid_result = defaultdict(list), defaultdict(list) # result 리스트
models = [] # 모델을 저장할 리스트

# k-fold
for idx,(train_idx, test_idx) in tqdm(enumerate(stratkfold.split(X,Y))):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]

    # 모델 선언
    model = _model.random_forest
    model.set_params(**{'max_depth': 19, 'max_leaf_nodes': 749, 'n_estimators': 349,'random_state':42})
    
    # 모델 훈련
    model.fit(x_train, y_train)
    models.append(model) # 훈련된 모델을 리스트에 추가
    train_pred, valid_pred = models[idx].predict(x_train), models[idx].predict(x_test)
    
    # store result
    for name, score in check_the_score(train_pred, y_train).items(): # train
        train_result[name].append(score)
    for name, score in check_the_score(valid_pred, y_test).items():  # valid
        valid_result[name].append(score)
# print score
print('----[K-Fold Train Score]-----')
score_print(train_result)
print('----[K-Fold Validation Score]-----')
score_print(valid_result)

5it [00:48,  9.63s/it]

----[K-Fold Train Score]-----
f1 score : 0.9215 / STD: (+/- 0.0022)
precision score : 0.8656 / STD: (+/- 0.0033)
recall score : 0.9852 / STD: (+/- 0.0017)
roc_auc_score score : 0.9866 / STD: (+/- 0.0009)
----[K-Fold Validation Score]-----
f1 score : 0.8166 / STD: (+/- 0.0072)
precision score : 0.7301 / STD: (+/- 0.0108)
recall score : 0.9264 / STD: (+/- 0.0059)
roc_auc_score score : 0.9513 / STD: (+/- 0.0030)


In [437]:
# threshold valid
threshold = 0.6
valid_result = defaultdict(list) # result 리스트
for idx, (_, test_idx) in tqdm(enumerate(stratkfold.split(X,Y))):
    x_test, y_test = X.iloc[test_idx], Y[test_idx]

    y_valid_pred = models[idx].predict_proba(x_test)[:,0]
    y_valid_pred = np.where(y_valid_pred >= threshold, False, True)
    
    for name, score in check_the_score(y_valid_pred, y_test).items():  # valid
        valid_result[name].append(score)
        
# print score
print('----[K-Fold Validation Score]-----')
score_print(valid_result)

2it [00:00,  9.71it/s]

[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42


4it [00:00, 10.15it/s]

[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42


5it [00:00,  9.90it/s]

----[K-Fold Validation Score]-----
f1 score : 0.8087 / STD: (+/- 0.0076)
precision score : 0.8788 / STD: (+/- 0.0092)
recall score : 0.7491 / STD: (+/- 0.0132)
roc_auc_score score : 0.8690 / STD: (+/- 0.0065)


In [486]:
# feature importance
feature_importances = model.feature_importances_
feature_names = x_train.columns
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_importance_dict = {k: v for k, v in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)}
for feature_name, importance in sorted_importance_dict.items():
    print(f"{feature_name}: {importance}")

lead_owner_converted_ratio: 0.5023084695182889
new_customer_idx: 0.23577397405783396
lead_desc_length: 0.04716885697346108
business_subarea: 0.028342316282734415
new_customer_type2: 0.02173172342044822
country: 0.019807551097815918
ver_win_rate_x: 0.019602276085594412
job_function: 0.018378823330094364
category_3: 0.012795084494969126
business_area: 0.012143367057762299
historical_existing_cnt: 0.011712618463274309
seniority_level: 0.011455350827059854
com_reg_ver_win_rate: 0.010406156150770788
category_2: 0.008063015615084525
new_customer_type1: 0.007116987731973587
customer_interest: 0.00601679961991281
bant_submit: 0.005505858528743072
new_expected_timeline: 0.004648156811388506
ver_win_ratio_per_bu: 0.0042648867281554
continent: 0.004211397819882915
less_timeline: 0.003517215694104122
business_unit: 0.0025387815219880617
new_inquiry_type: 0.001296388812249992
category_1: 0.0009955936014495095
enterprise: 0.00019834975495994225


## inference

In [587]:
test_drop = df_test.drop(['id','is_converted'],axis = 1)

In [588]:
# voting threshold
predicts = []
threshold = 0.9
for model in models:
    pred_proba = model.predict_proba(test_drop)[:,0]
    pred = np.where(pred_proba >= threshold, False, True)
    predicts.append(pred)

In [589]:
for i in predicts:
    print(sum(i))

1582
1649
1610
1610
1567


In [590]:
num_threshold = 1
pred = np.where( np.sum(predicts,axis = 0) >= num_threshold, 1, 0)
pred = np.where(pred == 0, False, True)
sum(pred)

1770

In [423]:
# difference
pred2 = pred  #pred1 718 | pred2 1142 | pred3 820
print(sum(pred))

1400


In [185]:
su = 0
for i,j in zip(pred3, pred1):
    if i != j:
        su +=1
su

249

## ensemble

In [591]:
pred4 = pred 
df_test['is_converted'] = pred

In [ ]:
date='0222'

In [492]:
pred1 # decision_tree: num_threshold: 1 + threshold: 0.9
df_test.to_csv(f"{date}_decision_tree.csv",index = False)

In [503]:
pred2 # xgb: num_threshold: 1, threshold = 0.9
df_test.to_csv(f"{date}_xgb.csv",index = False)

In [475]:
pred3 # lgb: num_threshold: 1 + threshold: 0.8
df_test.to_csv(f"{date}_lgb.csv",index = False)

In [592]:
pred4 # ramdom_forest: num_threshold: 1 + threshold: 0.9
df_test.to_csv(f"{date}_random_forest.csv",index = False)

In [597]:
df = pd.read_csv(f"{date}_cat.csv")
pred5 = np.array(df['is_converted'])# cat: num_threshold: 1 + threshold: 0.6

In [599]:
ensemble_ = np.array((pred1,pred2,pred3,pred4,pred5))

In [604]:
print(sum(pred1),sum(pred2),sum(pred3),sum(pred4),sum(pred5))

1682 1446 1609 1770 1424


In [634]:
ensemble_threshold = 3
pred = np.where( np.sum(ensemble_,axis = 0) >= ensemble_threshold, 1, 0)
pred = np.where(pred == 0, False, True)
sum(pred)

1523

## submission

In [630]:
df_test['is_converted'] = pred
df_test.to_csv("submission.csv",index = False)

In [622]:
sum(pred)

2075